In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=8, micro=2, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [7]:
# 求导
# 自定义一个函数表达式
def func(x):
    return 3. * x ** 2 + 2. * x -1

# 近似值求导法，传入被求导函数，x的值，近似精确度越小越接近
def near_derivation(f, x, eps=1e-5):
    return (f(x + eps) - f(x - eps)) / (2. * eps)

print(near_derivation(func, 1.))

8.000000000008


In [8]:
# 求偏导

def gunc(x1, x2):
    return (x1 + 6) * (x2 ** 2)

# 求偏导数时，将另一个未知数当作常量，只对指定目标使用近似求导即可
def near_derivation2(g, x1, x2, eps=1e-5):
    dg_x1 = near_derivation(lambda x: gunc(x, x2), x1, eps)
    dg_x2 = near_derivation(lambda x: gunc(x1, x), x2, eps)
    return dg_x1, dg_x2

print(near_derivation2(gunc, 2., 4.))

(15.999999999394275, 63.99999999828764)


In [11]:
# 使用tensorflow的求导接口

# 不指定persisten=True则tape只执行一次便关闭
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = gunc(x1, x2)

# 求偏导，直接传递z关于x1和x2的函数，以及求导目标未知量
dz_x1 = tape.gradient(z, x1)
print(dz_x1)

try:
    dz_x2 = tape.gradient(z, x2)
except RuntimeError as rune:
    print(rune)
# GradientTape.gradient can only be called once on non-persistent tapes.

tf.Tensor(9.0, shape=(), dtype=float32)
GradientTape.gradient can only be called once on non-persistent tapes.


In [13]:
# 使用tensorflow的求导接口

# 指定persisten=True
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as tape:
    z = gunc(x1, x2)

# 求偏导，直接传递z关于x1和x2的函数，以及求导目标未知量
dz_x1 = tape.gradient(z, x1)
dz_x2 = tape.gradient(z, x2)
print(dz_x1)
print(dz_x2)

# 指定persistent=True，要手动释放tape
del tape

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(48.0, shape=(), dtype=float32)


In [16]:
#同时求偏导，求导目标未知量作为列表参数传入
x1 = tf.Variable(2.0)
x2 = tf.Variable(3.0)
with tf.GradientTape() as tape:
    z = gunc(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=48.0>]


In [18]:
# 对constant常量求导，得到None
x1 = tf.constant(2.0)
x2 = tf.constant(3.0)
with tf.GradientTape() as tape:
    z = gunc(x1, x2)

dz_x1x2 = tape.gradient(z, [x1, x2])

print(dz_x1x2)

[None, None]


In [21]:
# 可以对多个目标函数的同一个变量求导，最终得到两结果之和
x1 = tf.Variable(5.0)
x2 = tf.Variable(4.0)

with tf.GradientTape() as tape:
    z1 = 3 * x1 + 2 * x2
    z2 = x1 ** 2 + 2 * x2

tape.gradient([z1, z2], [x1, x2])

[<tf.Tensor: shape=(), dtype=float32, numpy=13.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=4.0>]

In [26]:
# 求二阶导数
x1 = tf.Variable(2.0)
x2 = tf.Variable(99.0)

# 求导多次，需要指定persistent=True
with tf.GradientTape(persistent=True) as outer_tape:
    with tf.GradientTape(persistent=True) as inner_tape:
        z = gunc(x1, x2)
    inner_grads = inner_tape.gradient(z, [x1, x2])
outer_grads = [outer_tape.gradient(inner_grad, [x1, x2]) for inner_grad in inner_grads]
print(outer_grads)

[[None, <tf.Tensor: shape=(), dtype=float32, numpy=198.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=198.0>, <tf.Tensor: shape=(), dtype=float32, numpy=16.0>]]


In [23]:
del inner_tape
del outer_tape

In [29]:
# 模拟梯度下降
# 设置学习率
learning_rate = 0.1

# 定义变量x，初始化为0
x = tf.Variable(0.0)

for _ in range(100):
    with tf.GradientTape() as tape:
        z = func(x)
    dz_dx = tape.gradient(z, x)
    
    # 变量x减去倒数与学习率之积
    x.assign_sub(learning_rate * dz_dx)
    
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>


In [31]:
# 梯度下降与优化器结合使用
learning_rate = 0.1
x = tf.Variable(0.0)

# 随机梯度下降
optimizer = keras.optimizers.SGD(lr=learning_rate)

for _ in range(100):
    with tf.GradientTape() as tape:
        z = func(x)
    dz_dx = tape.gradient(z, x)
    optimizer.apply_gradients([((dz_dx, x))])
    
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.3333333>
